# Imports

In [1]:
from sys import path as syspath
syspath.append("..")

In [2]:
from MISalign.model.mis_file import MisFile,load_mis
from MISalign.canvas.canvas_solve import rectangular_solve
import MISalign.canvas.canvas_render as cr

# Render Setup

In [18]:
###
mis_filepath=r"..\example\data\set_a\mymis_calibrated.mis"
mis_project=load_mis(mis_filepath)
auto_save_renders=False
###


In [ ]:
relations=mis_project.get_rels('r')
image_names=mis_project.get_image_names()
origin=image_names[0]
origin_relative_offsets=rectangular_solve(
    relations=relations,
    image_names=image_names,
    origin=origin
)
print("Origin-relative offsets")
display(origin_relative_offsets)
image_filepaths=mis_project.get_image_paths()
image_sizes=cr.find_image_sizes(image_filepaths)
origin_relative_extents=cr.find_relative_extents(
    image_names=image_names,
    origin_relative_offsets=origin_relative_offsets,
    image_sizes=image_sizes)
canvas_extents, canvas_offsets=cr.resolve_extents(origin_relative_extents)
canvas_relative_offsets=cr.place_in_canvas(
    image_names,
    origin_relative_offsets,
    canvas_extents,
    canvas_offsets)
print("Canvas-relative offsets")
display(canvas_relative_offsets)


## Render Unblended

In [ ]:
unblended_canvas=cr.render_unblended(
    image_names,
    image_filepaths,
    image_sizes,
    canvas_relative_offsets,
    canvas_extents)
display(unblended_canvas)
if auto_save_renders:
    unblended_save_fp=mis_filepath.replace(".mis","-unblend.png")
    unblended_canvas.save(unblended_save_fp)

## Render Blended

In [ ]:
dfe_normalizer=cr.build_normalization(
    image_names,
    image_sizes,
    canvas_relative_offsets,
    canvas_extents,
    cr.weight_dfe)
blended_canvas_dfe=cr.render_blended(
    image_names,
    image_filepaths,
    image_sizes,
    canvas_relative_offsets,
    canvas_extents,
    cr.weight_dfe,
    dfe_normalizer)
display(blended_canvas_dfe)
if auto_save_renders:
    blended_save_fp=mis_filepath.replace(".mis","-blend.png")
    blended_canvas_dfe.save(blended_save_fp)

# Image Scale Bar
Note: You must still run the 1st cell to enable importing from MISalign.

In [2]:
from MISalign.calibration.scale_bar import image_with_scale_bar,scale_bar_calibrate,save_calibrated_image
from MISalign.calibration.calibrate import CalibrationManual
from PIL.Image import ROTATE_90,ROTATE_180,ROTATE_270,FLIP_LEFT_RIGHT,FLIP_TOP_BOTTOM
%matplotlib widget

In [4]:
## Select image, it can be either a filepath to an image or a PIL Image object.
###
selected_image=r"..\example\expected_result\set_a\a_mycanvas_blend.jpg"
# selected_image=blended_canvas_dfe
# selected_image=unblended_canvas
###

## Tranpose selected PIL Image object if needed.
# selected_image=selected_image.transpose(FLIP_LEFT_RIGHT)
# selected_image=selected_image.transpose(FLIP_TOP_BOTTOM)
# selected_image=selected_image.transpose(ROTATE_90)
# selected_image=selected_image.transpose(ROTATE_180)
# selected_image=selected_image.transpose(ROTATE_270)

###
# selected_calibration=mis_project.calibration
#
calibration_path=r"..\example\data\set_a\a_5x.miscal"
cm=CalibrationManual()
cm.load_calibration(calibration_path)
selected_calibration=cm.distances
###


In [ ]:
image_with_scale_bar(
    image=selected_image,
    scale_measurement="1mm",
    calibration=selected_calibration,
    loc="upper left")

In [8]:
scaled_dpi=1000
# Increasing this DPI makes the above image smaller
# As a result, increasing the DPI makes the scale bar text/box larger relative to the image.
# Changing this DPI does not effect the resolution of the final image
#
scale_bar_calibrate(scale_dpi=scaled_dpi)

In [40]:
if type(selected_image) is str:
    split_fp=selected_image.split(".")
    scalebar_save_fp=split_fp[0]+"-scale."+split_fp[1]
else:
    scalebar_save_fp=mis_filepath.replace(".mis","-scale.png")
save_calibrated_image(scalebar_save_fp,scale_dpi=scaled_dpi)